# Reference Suitability
This notebook is a tool for determining which standard materials are suitable for use as a fitting reference for an element.  The problem it is designed to address is interferences between the element and other elements in the material. It considers a default list of pure elemental and stoichiometric compound standard materials.  The characteristic X-ray associated with the specified element are divided into regions determined by the detector performance each of which will be fit independently.  The algorithm then checks each material containing the element of interest to determine which of these regions does not suffer an interference with any of the other elements in the material.

In [1]:
# Load the necessary libraries
using NeXLSpectrum
using DataFrames
# Configure tabulation
ENV["lines"], ENV["columns"] = 200, 400;


The performance of your detector is critical for determining how the peaks will be split up into independent regions each of which is fit independently.

In [2]:
# Build a detector model to match your EDS detector
# BasicEDS(channelCount, zeroOffset, channelWidth, FWHM_MnKa, low_level_discriminator, visibility)
det = BasicEDS(4096, -480.40409, 5.00525, 132.0, 110, 
  Dict( # What is the first element with visible KShell, LShell, ... peaks
    KShell=>n"Be",
    LShell=>n"Ca",
    MShell=>n"Cs",
    NShell=>n"Pu")
)

BasicEDS[4096 chs, E[ch] = -480.40409 + 5.00525⋅ch, 132.0 eV @ Mn K-L3, 110 ch LLD, [Be,Ca,Cs,Pu]]

Specify an element and load a default set of standard materials.  You can add additional materials if desired.

In [3]:
elm = parse(Element, "Fe")
stds=getstandards(elm, 0.01)
# push!(stds, parse(Material, "0.2*Au+0.8*Ag")) # Example of how to add a standard
asa(DataFrame, collect(stds))

,Material,N,O,F,Al,Si,P,S,Fe,Cu,Total
,String,Abstrac…,Abstrac…,Abstrac…,Abstrac…,Abstrac…,Abstrac…,Abstrac…,Abstrac…,Abstrac…,Abstrac…
1,Fe3N,0.0771554,0.0,0.0,0.0,0.0,0.0,0.0,0.922845,0.0,1.0
2,FeS2,0.0,0.0,0.0,0.0,0.0,0.0,0.534488,0.465512,0.0,1.0
3,Fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,CuFeS2,0.0,0.0,0.0,0.0,0.0,0.0,0.349406,0.304314,0.34628,1.0
5,FeP,0.0,0.0,0.0,0.0,0.0,0.356763,0.0,0.643237,0.0,1.0
6,Fe22+SiO4,0.0,0.694997,0.0,0.0,0.305003,0.0,0.0,1.0,0.0,2.0
7,Fe3Al2Si3O12,0.0,0.385718,0.0,0.108416,0.169275,0.0,0.0,0.336591,0.0,1.0
8,FeO,0.0,0.22269,0.0,0.0,0.0,0.0,0.0,0.77731,0.0,1.0
9,Fe3O4,0.0,0.276403,0.0,0.0,0.0,0.0,0.0,0.723597,0.0,1.0


The next cell performs all the useful work.  It takes an element, detector and the minimum mass-fraction and produces a table where the rows are the candidate materials and the columns are the fit regions.  A check mark indicates that the material is suitable for the region and an X indicates lack of suitability.  The ideal reference has all checks.  However, it is possible to use spectra from two or more materials to satisfy all the regions that need to be fit.

In [4]:
suitability(elm, stds, det)

,Material,Fe L3-M5 + 11 others,Fe K-L3 + 1 other,Fe K-M3 + 3 others
,Material,String,String,String
1,"Fe3N[N=0.0772,Fe=0.9228]",✗,✓,✓
2,"FeS2[S=0.5345,Fe=0.4655]",✓,✓,✓
3,Fe[Fe=1.0000],✓,✓,✓
4,"CuFeS2[Cu=0.3463,S=0.3494,Fe=0.3043]",✗,✓,✓
5,"FeP[P=0.3568,Fe=0.6432]",✓,✓,✓
6,"Fe22+SiO4[Fe=1.0000,Si=0.3050,O=0.6950]",✗,✓,✓
7,"Fe3Al2Si3O12[Al=0.1084,Fe=0.3366,Si=0.1693,O=0.3857]",✗,✓,✓
8,"FeO[Fe=0.7773,O=0.2227]",✗,✓,✓
9,"Fe3O4[Fe=0.7236,O=0.2764]",✗,✓,✓
